# Ingesting a CSV File

Let's look at another example, where we both want to deserialize and validate data - reading from a CSV file.

CSV files do not have data types built-in - everything is purely a string, so we can use Pydantic to also coerce/convert the data as needed for our application needs.

In this directory, you will find a sample CSV file: `pop_estimates.csv`.

Let's inspect this file:

In [1]:
import csv

with open("pop_estimates.csv") as f:
    data = csv.reader(f)
    for _ in range (5):
        print(next(data))

['Geographic Area', 'July 1, 2001 Estimate', 'July 1, 2000 Estimate', 'April 1, 2000 Population Estimates Base']
['United States', ' 284,796,887 ', ' 282,124,631 ', ' 281,421,906 ']
['Alabama', ' 4,464,356 ', ' 4,451,493 ', ' 4,447,100 ']
['Alaska', ' 634,892 ', ' 627,601 ', ' 626,932 ']
['Arizona', ' 5,307,331 ', ' 5,165,274 ', ' 5,130,632 ']


As you can see the first row has column names, but if you look at the population estimate data, they look like this:
```
' 284,796,887 '
```

Now Pydantic is good, but not that good! Clearly we'll have to create a custom validator that we can use to parse values like the one above into proper integers when deserializing.

In [2]:
def name_int(value: str):
    try:
        return int(value.strip().replace(",", ""))
    except Exception as ex:
        raise ValueError("data could be parsed into a valid integer")

Let's make an annotated field out of this that will incorporate this deserializer. In this case, we need to make the validator a **before** validator - since we want out field to be an `int`, if we use an **after** validator, then we'll have issues because Pydantic cannot parse those values to int.

In [3]:
from typing import Annotated

from pydantic import BeforeValidator


FunkyInt = Annotated[int, BeforeValidator(name_int)]

And now let's create our model:

In [4]:
from pydantic import BaseModel

In [5]:
class Estimate(BaseModel):
    area: str
    july_1_2001: FunkyInt
    july_1_2000: FunkyInt
    april_1_2000: FunkyInt

We're going to read the CSV data using `csv.DictReader` - the advantage of this is that we'll get a dictionary for every row, with the keys being the field names, so it will be easy to pass to `model_validate()`.

Let's just see how that works before we use our model:

In [6]:
with open("pop_estimates.csv") as f:
    data = csv.DictReader(f, fieldnames=["area", "july_1_2001", "july_1_2000", "april_1_2000"])
    for _ in range (5):
        print(next(data))

{'area': 'Geographic Area', 'july_1_2001': 'July 1, 2001 Estimate', 'july_1_2000': 'July 1, 2000 Estimate', 'april_1_2000': 'April 1, 2000 Population Estimates Base'}
{'area': 'United States', 'july_1_2001': ' 284,796,887 ', 'july_1_2000': ' 282,124,631 ', 'april_1_2000': ' 281,421,906 '}
{'area': 'Alabama', 'july_1_2001': ' 4,464,356 ', 'july_1_2000': ' 4,451,493 ', 'april_1_2000': ' 4,447,100 '}
{'area': 'Alaska', 'july_1_2001': ' 634,892 ', 'july_1_2000': ' 627,601 ', 'april_1_2000': ' 626,932 '}
{'area': 'Arizona', 'july_1_2001': ' 5,307,331 ', 'july_1_2000': ' 5,165,274 ', 'april_1_2000': ' 5,130,632 '}


So, only thing is we need to skip the first row, since those are the column headers (which we neither need, nor use).

We'll also make a generator function as it will make it easier to encpasulate the data reading part.

In [7]:
def estimates():
    with open("pop_estimates.csv") as f:
        data = csv.DictReader(f, fieldnames=["area", "july_1_2001", "july_1_2000", "april_1_2000"])
        next(data)  # skip header row

        for row in data:
            yield Estimate.model_validate(row)

We can either just iterate through the data:

In [8]:
for estimate in estimates():
    print(estimate)

area='United States' july_1_2001=284796887 july_1_2000=282124631 april_1_2000=281421906
area='Alabama' july_1_2001=4464356 july_1_2000=4451493 april_1_2000=4447100
area='Alaska' july_1_2001=634892 july_1_2000=627601 april_1_2000=626932
area='Arizona' july_1_2001=5307331 july_1_2000=5165274 april_1_2000=5130632
area='Arkansas' july_1_2001=2692090 july_1_2000=2678030 april_1_2000=2673400
area='California' july_1_2001=34501130 july_1_2000=34000446 april_1_2000=33871648
area='Colorado' july_1_2001=4417714 july_1_2000=4323410 april_1_2000=4301261
area='Connecticut' july_1_2001=3425074 july_1_2000=3410079 april_1_2000=3405565
area='Delaware' july_1_2001=796165 july_1_2000=786234 april_1_2000=783600
area='District of Columbia' july_1_2001=571822 july_1_2000=571066 april_1_2000=572059
area='Florida' july_1_2001=16396515 july_1_2000=16054328 april_1_2000=15982378
area='Georgia' july_1_2001=8383915 july_1_2000=8229823 april_1_2000=8186453
area='Hawaii' july_1_2001=1224398 july_1_2000=1212281 apr

or we could make it into a list and work with individual elements:

In [9]:
data = list(estimates())

In [10]:
data[0]

Estimate(area='United States', july_1_2001=284796887, july_1_2000=282124631, april_1_2000=281421906)